# Am I The A-hole Predictor Project

## Table of contents
- <a href="#intro">Introduction</a>
- <a href="#loading">Loading and assessment</a>
- <a href="#cleaning">Cleaning</a>
- <a href="#overview">General Overview</a>
- <a href="#insights">Insights</a>
- <a href="#tester">Predictions Tester</a>
- <a href="#discussion">Discussion</a>

<a id="intro"></a>
## Introduction
This project is based on one of Reddit's most popular section: <a href="https://www.reddit.com/r/AmItheAsshole/">r/AmITheAsshole</a>. This subreddit (AITA for short) has a simple premise of having people post about social experiences where they're unsure if they're the one at fault or not. Redditors participate in deciding their judgement, commenting on the situation, giving a brief summary of what they think is going on and argumenting for each of the valid judgements:

- **YTA** (You're the A-hole)
- **NTA** (Not the A-hole, but the other person is)
- **ESH** (Everyone Sucks Here)
- **NAH** (No A-holes Here)

People vote in their comments, and after 18 hours, a flair is given to each post with the verdict of the public, which then usually translates to the original posters reacting to their judgement, for better or worse (and this is where this subreddit gets one of the juiciest comment sections on the entire website!). However, this being both Reddit and the internet, there are some biases included, and the crowd allegedly tends to pick certain verdicts when the story has some buzzwords or characters which are not as favored in the internet, such as: mothers-in-law, pregnant people, childen being loud, among many others, as well as certain situations where the subreddit is pretty vocal about defending, specially when it includes people asking for help when there's no legal obligation to do so.

In this project, I took this perceived notion of being able to predict the outcome when the story is told in a certain way or with certain participates, and retrieved a dataset of 150,000 submissions, processed to obtain some descriptive statistics about posts based on Reddit activity such as comments and up/downvotes to characterize the posts and see if the usually talked about patterns are actually there or not, and build a machine learning model to predict the verdict of a user input text telling a story.

**Import statements**

In [ ]:
import pandas as pd
import datetime
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
%matplotlib inline

<a id="loading"></a>
## Loading and assessment

The dataset was obtained from a script which uses two Reddit APIs to obtain posts and then pull information about each of them.

In [ ]:
df = pd.read_csv(os.path.join("reddit_scraper/reddit_posts.csv"))

In [ ]:
df.head()

In [ ]:
df.info()

The fields available were decided based on the <a href="https://praw.readthedocs.io/en/latest/code_overview/models/submission.html?highlight=submission">PRAW docs</a> for both the ML model and analytics:
- **title**: title of the submission
- **id**: ID of the submission
- **score**: The number of upvotes for the submission.
- **upvote_ratio**: The percentage of upvotes from all votes on the submission.
- **url**: The URL the submission links to, or the permalink if a selfpost.
- **num_comments**: The number of comments on the submission.
- **body (renamed from selftext)**: The submissions’ selftext - an empty string if a link post. 
- **created**: Time the submission was created, represented in Unix Time.
- **edited**: Whether or not the submission has been edited.
- **verdict** (renamed from link_flair_text): The link flair’s text content, or None if not flaired.
- **over_18**: Whether or not the submission has been marked as NSFW.

The main fields for the model are the content body of the submission and the verdict, which comes from the submission flair, which is given after the 18 hours of people commenting on the post.

<a id="cleaning"></a>
## Cleaning

For the analysis, we will remove posts with tags outside the verdicts (mod posts or without a flair). Also, "Not enough info" posts will be removed as well, as they're considered invalid from the submission point of view.

The edited column has a mix of datatypes, so we'll change it to a boolean type to know if the post was edited or not.

After cleaning, over 75% of the posts were removed, so the clean dataframe has around valid 35k posts.

In [ ]:
df.verdict.value_counts()

In [ ]:
nan = np.nan
clean_df = df.query("verdict not in ['TL;DR', 'UPDATE', @nan, 'Talk ENDED', 'Open Forum', 'Mods Needed!', 'META', 'Not enough info']")

In [ ]:
is_edited = []

for i in clean_df.edited.values:
    if i == 'False':
        column_value = False
    else:
        column_value = True
    is_edited.append(column_value)

In [ ]:
clean_df.insert(8, 'is_edited', is_edited)

In [ ]:
clean_df = clean_df.drop(columns=["edited"])

In [ ]:
clean_df.created = pd.to_datetime(clean_df.created)

In [ ]:
clean_df.insert(8, 'isoweek', clean_df.created.dt.isocalendar().week)

In [ ]:
clean_df.insert(8, 'hour_of_day', clean_df.created.dt.hour)

In [ ]:
clean_df.info()

<a id="overview"></a>
## General overview

We start by getting a general overview of the valid posts, which were defined as having a verdict flair and a body that wasn't removed or deleted. The dataset is unbalanced, as it has a lot more submissions where the verdict was **NTA** over all the other verdicts. This already gives us indication that people tend to post more stories where they wouldn't be deemed the A-holes (a popular theory in the subreddit) and/or that people tend to validate people in their actions, which could also be the case since they're posting one-sided stories where they could always hide inconvenient facts.

The timeframe for the data is from mid-August to end of year, 2021, which covers some major holidays, so we could take a closer look at holiday-related posts and see if there's an increase during these days.

In [ ]:
clean_df.verdict.value_counts()

In [ ]:
clean_df.sort_values(by="created")

<a id="insights"></a>
## Insights

As previously noted, most of the posts have a **NTA** verdict, and account for nearly 75% of the cleaned dataset. We'll explore the distribution of different attributes grouped by verdict to see if we can find some additional insights as to how people behave when interacting with this subreddict.

In [ ]:
total_posts = clean_df.id.count()
pctg_verdicts = clean_df.verdict.value_counts() / total_posts * 100

In [ ]:
## general parameters for plotting
legends = list(clean_df.verdict.unique().astype("str"))
ind = np.arange(len(pctg_verdicts))
ticks = np.arange(len(legends))
width = 0.9

In [ ]:
## plotting
plt.figure(figsize=(8,5.5), dpi=100)
plt.bar(ind, pctg_verdicts, width, label = "Percentage of posts", color=(0.33, 0.62, 0.67, 0.85))

plt.ylabel('Percentage of posts', fontsize=14)
plt.xlabel('Verdicts', fontsize=14)
plt.title('Proportion of verdicts', fontsize=20)
locations = ticks
plt.xticks(locations, legends, fontsize=12)
plt.yticks()
sns.despine();

### Removed and deleted posts proportion

A very usual concern when posting is being found by people you know on Reddit, or sometimes you post something and get hit with the overwhelming feel of shame that you want to delete your existence off the internet (though if it's on the Internet, it'll stay there forever!), so you try to at least clear your Reddit traces. Sometimes, your content doesn't fulfill subreddit standards and mods delete your post, to avoid clutter. There might be a few reasons, but the reality is that a lot of posts get removed by users or mods, so we tried to take a closer look at this missing posts, using both the original dataframe. 

When checking, we find over 100k posts (which were removed from the original dataset for other analysis) which don't have neither a flair or body. The removed body posts are invalid submissions cleaned by mods or automods. The deleted submissions represents those removed by the original poster, which are a much less in comparison.

The proportions of deleted posts are similar for all verdicts and slightly higher for the more positive verdicts, so apparently people who are deemed A-holes by Redditors are not into removing their content after all!

In [ ]:
df.query("verdict.isnull() & body in ['[removed]', '[deleted]']",engine='python').body.value_counts()

In [ ]:
total_posts = clean_df.verdict.value_counts()
deleted_posts = clean_df.query("body == '[deleted]'").verdict.value_counts()
total_posts / deleted_posts

### Time distribution of posts

When looking at distribution of valid posts based on when they were posted, we some clear find some patterns. First, the distribution based on hour of day (UTC) reached its peak during afternoon hours, and slowly decreases until its lowest point at 6AM. Even when considering timezones, we can see that people are posting a lot more during school/work hours rather than nighttime, which could have been expected if we consider the typical night-owl stereotype. 

For week distribution, the time range considered is from late August to end of year, so we'd cover Thanksgiving (ISO Week 47) and Christmas (ISO Week 51), which are indeed the busiest weeks in our range, and matches with the usual background stories for AITA posts.

In [ ]:
hour_distribution = clean_df.groupby('hour_of_day').id.count().values

In [ ]:
## general parameters for plotting
legends = clean_df.groupby('hour_of_day').id.count().index
ind = np.arange(len(hour_distribution))
ticks = np.arange(len(legends))
width = 0.65

In [ ]:
## plotting
plt.figure(figsize=(10,5.5), dpi=100)
plt.bar(ind, hour_distribution, width, label = "Percentage of posts", color=(0.33, 0.62, 0.67, 0.85))

plt.ylabel('Amount of valid posts', fontsize=14)
plt.xlabel('Hour of day', fontsize=14)
plt.title('Hour of Day Distribution', fontsize=20)
locations = ticks
plt.xticks(locations, legends, fontsize=12)
plt.yticks()
sns.despine();

In [ ]:
week_distribution = clean_df.groupby('isoweek').id.count().values

In [ ]:
## general parameters for plotting
legends = clean_df.groupby('isoweek').id.count().index
ind = np.arange(len(week_distribution))
ticks = np.arange(len(legends))
width = 0.8

In [ ]:
## plotting
plt.figure(figsize=(10,5.5), dpi=100)
plt.bar(ind, week_distribution, width, label = "Percentage of posts", color=(0.33, 0.62, 0.67, 0.85))

plt.ylabel('Amount of valid posts', fontsize=14)
plt.xlabel('ISO Week', fontsize=14)
plt.title('ISO Week Distribution', fontsize=20)
locations = ticks
plt.xticks(locations, legends, fontsize=12)
plt.yticks()
sns.despine();

### Number of comments and upvotes

When looking at descriptive statistics of the numerical fields, we can see that, in average, posts with YTA and ESH verdicts have higher number of comments than NTA and NAH, which could be due to people wanting to voice their opinions more on situations where you want to tell the OPs they're wrong, or because longer discussions start in the comment sections. On a similar note, the scores and upvote ratios are lower in YTA than the rest of the verdicts, which is also related to the previous point, as people will downvote posts where people are being AHs.

In [ ]:
clean_df.groupby("verdict").describe().transpose()

In [ ]:
num_comments = clean_df.groupby("verdict").num_comments.mean()

In [ ]:
## general parameters for plotting
legends = upvote_ratio.index
ind = np.arange(len(upvote_ratio))
ticks = np.arange(len(legends))
width = 0.7

In [ ]:
## plotting
plt.figure(figsize=(10,5.5), dpi=100)
plt.bar(ind, num_comments, width, label = "Comments", color=(0.33, 0.62, 0.67, 0.85))

plt.ylabel('Average comments', fontsize=14)
plt.xlabel('Verdict', fontsize=14)
plt.title('Average comments per post', fontsize=20)
locations = ticks
plt.xticks(locations, legends, fontsize=12)
plt.yticks()
sns.despine();

### Word Clouds

As an extra, and to compare the most used words in our two opposite sides: YTA and NTA verdicts, we'll create some word clouds images for both classes, and see what we can find. First, we'll create the files necessary for the script, running the following cells, and then we can run the script from the main folder, running `python wordclouds/wordcloud_builder.py`

In [ ]:
nta_df = clean_df.query("verdict == 'Not the A-hole'").sort_values(by="num_comments", ascending=False).head(500)

In [ ]:
yta_df = clean_df.query("verdict == 'Asshole'").sort_values(by="num_comments", ascending=False).head(500)

In [ ]:
nta_text = []

for text in nta_df.body.values:
    with open(os.path.join("wordclouds/nta_text.txt"), "a+") as f:
        f.write(text)

In [ ]:
yta_text = []

for text in yta_df.body.values:
    with open(os.path.join("wordclouds/yta_text.txt"), "a+") as f:
        f.write(text)

<a id="tester"></a>
## Predictions Tester

After building our model in previous steps (see instructions on this repo's README file), we can test how it works with some real life examples. Recent posts were taken from Reddit and used as test for each of the verdicts. The predictions were NTA for 3/4 cases (including the NTA test), and the ESH test ended as **YTA**, which was the only one classified as deeming the OP as an A-hole. 

You can also test your own story if you want to at the end of this section!

In [ ]:
def tokenize(text):
    
    lemmatizer = WordNetLemmatizer()

    urls = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    detected_urls = re.findall(urls, text)

    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    words = word_tokenize(text)
    stop_words = stopwords.words("english")
    words = [x for x in words if x not in stop_words]

    clean_tokens = []
    for word in words:
        clean_token = lemmatizer.lemmatize(word).strip().lower()
        clean_tokens.append(clean_token)

    return clean_tokens

In [ ]:
model = joblib.load("model/model.pkl")

In [ ]:
#nta
nta_test = "I dont even know where to begin. My kids (9m, 9f, 6m) hardly ever see their dad or his family. He stopped taking the kids randomly back in 2017 and after that point, the kids grandmother would randomly take them but it was incredibly sporadic. Like once every 6 months or so. When she would take them, they would see their dad and most of the family- because usually their grandmother only took them during holiday. I was very lenient with this. The kids wanted to go and I wasnt about to tell them no, despite me missing every single holiday with my kids since 2017 because of this schedule. But this is a selfish thought on my end anyways so I overlooked it. The kids had fun. That's all that matters.So as of recently I noticed that their grandmother and even their dad started taking them more. Initially I assumed it was because their dad finally came to his senses but apparently that's not the case. But regardless, like once every couple of months the kids have been going there. Well, their grandmother got ahold of me about 3 weeks ago and told me that my children's grandfather had a heart attack and needed surgery but assured me he was fine. If he hadnt been, I would have 100% told the kids myself. A few days later, the grandmother posted a video on TikTok stating she had been diagnosed with Alplacia and then on Thursday I'm told that the kids great grandfather had been diagnosed with stage 4 lung cancer. I didnt tell my kids about any of it. I had plenty of reasons not to. One being the fact that the kids were going over soon so I figured it be best that they be the ones to break the news and two being that I knew my kids would completely panic and start freaking out and ask to go see them right then and there- which would have been denied by the grandparents due to them having a busy schedule. So I just didnt say anything.Well, my kids go over there last weekend and when they get dropped off, their grandmother asks me why I hadnt told them what was going on and stated that it ''took away from their time' with the kids because they had to explain everything, which led to my kids panicking and asking questions all weekend versus have a good time. She said I should have told them and the fact that I didnt really ruined their weekend. AITA?"

In [ ]:
nah_test = "Today is my birthday!My family and I don’t see each other a lot so usually birthdays are when we all come together. I haven’t heard from my Mom or grandma at all until last Saturday.My grandma asked me what I wanted to do and I told her I didn’t know if anyone wanted to do anything because nobody had said anything. She then told me I should’ve said something sooner because they were waiting on me.That took me by surprise because on Facebook, I always see them eating together with my uncle who lives out of town. I never get invited and I should probably speak up about my feelings but haven’t yet. I just feel like they’re trying to pick fights with me because other than them, my friends, other grandma and even my fiancés family have asked me what my plans are and we’ve already celebrated."

In [ ]:
#yta_test = "Using my alt account.Last Friday, I (25M) was killing time at work with my friend “Cap” (24M). We have one of those big industrial scanners, and he suggested me to scan my ass and see what happens. I agreed to try, and I balanced on a chair to put my ass on the machine. It didn’t work, so Cap told me to pull down my pants and boxers.Just as this was happening, my team leader “Sarah” (30?F) came into the print room. She just walked out without a word, but I heard from Cap that she is accusing me of making work into a sexual environment. I spoke to her kindly and told her it wasn’t a lood thing and was only a joke.Now, she’s blowing up our team Skype with all kinds of allegations about me, including that I have a serious disease. Not sure if I can name it here, but I’m sure you can guess. I’m going to HR tomorrow about her, but AITA for goofing around with my friend in the first place?"
yta_test = "I (30f) have a younger brother Michael (24m) who got an apartment with his girlfriend (22f) of 5yrs a year ago. I went to visit and before I even walked in I could hear her barking orders at him to clean this, do that, etc. After I came in and we all sat down I asked what was all that I heard. They both looked at me confused and I explained I heard her saying all that stuff to him. She laughed out off saying 'oh that's because Micheal won't do anything unless I act like his mother.' I was taken back by her casually putting down my brother like that . After we ate and she went to wash dishes I pulled him to aside to ask why he just accepted the way she treats him. At first he tried to make excuses saying 'she is just a traditional girl and she likes them having set roles and jobs in the household.' I told him 'that is bs and this is 2022 relationships are supposed to be 50/50 or nothing at all.' I also told him it wasn't okay for her to be jobless and just staying home all day is childish and they need to have a major talk.' I know he took my words to heart because a few days later I found out from my younger sister they got into multiple heated arguments for few days after then she left.Weeks later I thought everything was said and done till I got an angry call from my mom telling me to fix what I ruined and to go back to my brother's place. I went over there oh dear Lord the smell as soon as you open the door is awful. It was a mix of dirty dishes mixed with clothes on the floor and furniture, dried up sticky marks on the carpet and tiles from stuff spilled and left there, and trash piled up near the full trash can. He didn't even react to me seeing all of this and just sat back down kept playing his game. I asked him how has he been since she left and he claims he is fine except for shopping and cooking meals. I talked with him a little bit more and he seemed fine. I left and then called my mother to say 'he was fine. A complete slob but fine.' She yelled at me saying how 'I shouldn't have gotten involved because now he has nobody to take care of him and he doesn't have the drive to do it himself outside of basic needs.' I told her 'it's not my fault she failed as mom to teach him how to take care of himself and he is grown man he gets to decide how he wants to live.' I hung up and I've been receiving calls and texts from her and a few other family members saying how I shouldn't have interfered. Yesterday my fiance told me his friend who also lives in the same apartment complex talked to the landlord and he is thinking about kicking him out due to the indecent smell coming from his place (he hasn't seen the inside) and he isn't allowed to stay with us if he gets kicked.Edit: to clarify she comes from a traditional family so she was planning to be SAHM/SAHW so she was okay with him with till he asked her for 50/50 and she gets a job too."

In [ ]:
esh_test = "During the middle of February, my girlfriend (20F) ended things with me (21M). We kept hanging out every day because I was one of her only friends and she still wanted a friendship.I was pretty pissed when it happened and texted my friends “Eff gf for life.” I also kissed 2 people a couple days after she ended things. The last thing I did was talk about how hot other girls were, one of the girls was her friend.My gf also kissed another guy 2.5 weeks after it happened. She was planning on sleeping with him but they never got the chance before I found out. I was pretty pissed but let it go after I found out they hadn’t slept together because I had done the same and didn’t tell her either and we were broken up.My buddy’s gf went on his phone and found the texts talking about making out with other girls, calling other girls hot, and venting about my gf to my best friend. She sent my gf screenshots of the texts and ever since then she has been furious with me. She says I betrayed her on a Relationship level and friendship level. Am I the asshole for making out with girls and ranting about my gf after she ended things. I know it was mean to talk poorly about her but I know she has done the same. Reddit what do you think?"

In [ ]:
print(f"The prediction for the NTA test is: {model.predict([nta_test])[0]}")
print(f"The prediction for the NAH test is: {model.predict([nah_test])[0]}")
print(f"The prediction for the YTA test is: {model.predict([yta_test])[0]}")
print(f"The prediction for the ESH test is: {model.predict([esh_test])[0]}")

You can try your own story, running the following cell:

In [ ]:
your_story = "" ## write your story here

print(f"The prediction for your story is: {model.predict([your_story])[0]}")

<a id="discussion"></a>
## Discussion

This project represents the end of the Nanodegree, and was created from scratch thinking of something that could be both fun and could provide insight from a popular website that I personally browse everyday. It was challenging, and tries to cover a little bit of everything learned through this course. 

The dataset was obtained from scratch using Reddit APIs, which was a whole world to explore on its own and took a lot of testing to get the information with the desired format and using the right filters. Thanks to this, the dataset required minimal cleaning steps, mostly for formatting issues, as it was designed and then retrieved from the API with the project in mind. This process required some iteration regarding the size of the dataset, because over 3/4 of the original posts were removed as they had an invalid or null flair, so the script was modified until a better sample size was gathered for the smaller classes.

The machine learning pipeline was similar to what was covered during the course, but proved to be even more rewarding when using original data and iterating through the script to improve its metrics for a better model. The original dataset was biased towards a particular verdict, so several things were tested before the final version which improved its metrics from the first one. A few things were implemented, like accounting for imbalaced classes with the `class_weight` parameter, trying different dataset sizes, trying two classifiers (AdaBoostClassifier and RandomForestClassifier) and looking through different parameters to test with GridSearch. The final F-score considering all implementations reached 0.6, which improved from the previous trials (AdaBoostClassifier gave the lowest F-score of all tests). This metric was selected as the best one to measure model performance, as we were dealing with unbalanced classes. It's not perfect (yet), due to the complexity of this challenge, which might be a bit ambitious in wanting to classify based on internet users' moral compass, which is an issue on its own.

As for analytical results, we were able to visualize a few different trends regarding behaviour around these posts, like the comment activity and willingness to delete your own posts based on verdicts. We also saw some pattern when it comes to distribution of posts for hours of day and ISO weeks. The word clouds bring us insight about the most common words between two opposite verdicts, and might be interesting to analyze when it comes to the popular topics talked about in these two classes.

This project helped consolidate the notions of how to approach a data science problem with a real-life example, with data obtained from scratch and processed entirely for this purpose, and build a model that could turn into something fun to use, as well as learn the difficulties when working with text data and trying to interpret moral decisiones based on words. As for things which could be further improved, a larger dataset could be retrieved to get more examples of the smaller classes, and try additional NLP algorithms for text analysis. A web app could be built with the model in the backend, to directly give a story as an input and receive a prediction instead of using this notebook.